In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models

: 

In [ ]:
# Data processing
# Load the CSV data
df = pd.read_csv('cleaned_output.csv')

# Preprocessing - Flatten 'lo' column (list of tuples)
# We'll convert 'lo' from string format to actual list of tuples
def parse_lo(lo_str):
    # Safely evaluate the stringified tuple list (this converts it to a list of tuples)
    try:
        lo_list = ast.literal_eval(lo_str)
        return [tup[1] for tup in lo_list]  # We take the second element of each tuple
    except Exception as e:
        print(f"Error parsing lo: {e}")
        return []

df['lo'] = df['lo'].apply(parse_lo)

# Now, we'll split 'lo' into separate columns for each tuple value
lo_columns = ['lo_' + str(i) for i in range(df['lo'].apply(len).max())]  # Dynamically create columns for the lo list

# Expand the 'lo' column into multiple columns
df[lo_columns] = pd.DataFrame(df['lo'].to_list(), index=df.index)

# Drop the original 'lo' column (since we now have separate columns)
df.drop(columns=['lo'], inplace=True)

In [ ]:
# Model training

# Define features (X) and target (Y)
X = df.drop(columns=['c_0', 'c_1', 'b_0', 'b_1', 'q_0', 'q_1'])
y = df[['c_0', 'c_1', 'b_0', 'b_1', 'q_0', 'q_1']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the Neural Network Model
model = models.Sequential()

# Input Layer (taking all the features)
model.add(layers.InputLayer(input_shape=(X_train_scaled.shape[1],)))

# Hidden Layers (2 layers with 64 units and ReLU activation)
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

# Output Layer (6 values: c_0, c_1, b_0, b_1, q_0, q_1)
model.add(layers.Dense(6))  # No activation because we are doing regression

# Compile the model
model.compile(optimizer='adam', loss='mse')  # MSE loss for regression tasks

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)



In [ ]:
# Evaluate the model on the test set
loss = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss (MSE): {loss}")

# Make predictions
y_pred = model.predict(X_test_scaled)

# You can now use `y_pred` to compare against `y_test` or save the model